# Importing libraries

In [11]:
from collections import defaultdict
import os
import h5py
import numpy as np
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
from pathlib import Path
import zipfile
from utils import load_fmri, align_features_and_fmri_samples, align_features_and_fmri_samples_friends_s7


# Dataset Class for Stimulus-fMRI Alignment

A dataset class that aligns stimulus feature embeddings with fMRI data.

## Parameters

- **features_dir** : *str*
    Directory containing stimulus features from the HuggingFace dataset
- **fmri_dir** : *str*
    Directory containing fMRI data from the Algonauts 2025 competition  
- **movies** : *list*
    List of movies to use for training
- **subject** : *int*
    Subject ID number
- **excluded_samples_start** : *int, default=5*
    Number of samples to exclude from the start
- **excluded_samples_end** : *int, default=5*
    Number of samples to exclude from the end
- **hrf_delay** : *int, default=3*
    Hemodynamic response function delay
- **stimulus_window** : *int, default=5*
    Size of the stimulus window


In [12]:
class AlgonautsDataset(Dataset):
    def __init__(self, features_dir, fmri_dir, movies, subject, excluded_samples_start=5, excluded_samples_end=5, hrf_delay=3, stimulus_window=5):
        self.features_dir = features_dir
        self.fmri_dir = fmri_dir
        self.movies = movies
        self.subject = subject
        self.excluded_samples_start = excluded_samples_start
        self.excluded_samples_end = excluded_samples_end
        self.hrf_delay = hrf_delay
        self.stimulus_window = stimulus_window
        self.partition_indices = defaultdict(list)
        
        # First load all raw features
        stimuli_features = {"visual": {}, "audio": {}, "language": {}}
        # Load audio and video features first
        for movie in self.movies:
            if 'friends' in movie:
                season = movie.split('-')[1]
                dir_list = sorted(os.listdir(self.features_dir + 'audio')) #List of all audio for each subset of dataset
                for episode in dir_list:
                    if f"{season}e" in episode and '_features_' in episode:
                        episode_base = episode.split('_features_')[0] # friends_s01e01 and so on....
                        
                        for modality in ['audio', 'visual']:
                            with h5py.File(os.path.join(self.features_dir, modality, f"{episode_base}_features_{modality}.h5"), 'r') as f:
                                try:
                                    stimuli_features[modality][episode_base.split('_')[1]] = f[episode_base.split('_')[1]][modality][:]
                                except:
                                    f.visit(lambda x: print(x))
                lang_dir_list = sorted(os.listdir(self.features_dir + 'language'))
                for episode in lang_dir_list:
                    if f"{season}e" in episode and '_features_' in episode:
                        episode_base = episode.split('_features_')[0]
                        
                        with h5py.File(os.path.join(self.features_dir, 'language', f"{episode_base}_features_language.h5"), 'r') as f:
                            try:
                                st_season_episode = episode_base.split('_')[1]
                                stimuli_features['language'][st_season_episode] = f[st_season_episode]['language_pooler_output'][:]
                            except:
                                f.visit(lambda x: print(x))
            else:
                movie_name = movie.replace('movie10-', '')
                partitions = sorted([f for f in os.listdir(self.features_dir + 'audio') if movie_name in f and '_features_' in f])
                
                for partition in partitions:
                    partition_base = partition.split('_features_')[0]
                    
                    for modality in ['audio', 'visual']:
                        with h5py.File(os.path.join(self.features_dir, modality, f"{partition_base}_features_{modality}.h5"), 'r') as f:
                            try:
                                stimuli_features[modality][partition_base] = f[partition_base][modality][:]
                            except:
                                f.visit(lambda x: print(x))
                lang_partitions = sorted([f for f in os.listdir(self.features_dir + 'language') if movie_name in f and '_features_' in f])
                
                for partition in lang_partitions:
                    partition_base = partition.split('_features_')[0]
                    
                    with h5py.File(os.path.join(self.features_dir, 'language', f"{partition_base}_features_language.h5"), 'r') as f:
                        try:
                            stimuli_features['language'][partition_base] = f[partition_base]['language_pooler_output'][:]
                        except:
                            f.visit(lambda x: print(x))

        fmri_data = load_fmri(self.fmri_dir, self.subject)
        self.raw_stimuli = stimuli_features

        self.aligned_features, self.aligned_fmri = align_features_and_fmri_samples(
            stimuli_features, 
            fmri_data, 
            self.excluded_samples_start, 
            self.excluded_samples_end, 
            self.hrf_delay, 
            self.stimulus_window, 
            self.movies
        )

    def __len__(self):
        return self.aligned_features['audio'].shape[0]

    def __getitem__(self, idx):
        return {
            'audio': self.aligned_features['audio'][idx],
            'video': self.aligned_features['visual'][idx],
            'language': self.aligned_features['language'][idx],
            'fmri': self.aligned_fmri[idx]
        }
    
    def get_raw_stimuli(self):
        return self.raw_stimuli
   

In [13]:
features_dir = '/home/pranav/mihir/algonauts_challenge/AlgonautsDS-features/developer_kit/stimulus_features/raw/'
fmri_dir = '/home/pranav/mihir/algonauts_challenge/algonauts_2025.competitors/fmri/'
# movies_train = ["friends-s01"]
movies_train = ["friends-s01", "friends-s02", "friends-s03", "friends-s04", "friends-s05", "movie10-bourne", "movie10-figures", "movie10-life", "movie10-wolf"]
movies_val = ["friends-s06"]
modality = "all"  #@param ["visual", "audio", "language", "all"]

excluded_samples_start = 5  #@param {type:"slider", min:0, max:20, step:1}
excluded_samples_end = 5  #@param {type:"slider", min:0, max:20, step:1}
hrf_delay = 3  #@param {type:"slider", min:0, max:10, step:1}
stimulus_window = 5

subject = 1 #@param ["1", "2", "3", "5"] {type:"raw", allow-input: true}

train_ds = AlgonautsDataset(features_dir, fmri_dir, movies=movies_train, subject=subject, excluded_samples_start=excluded_samples_start, excluded_samples_end=excluded_samples_end, hrf_delay=hrf_delay, stimulus_window=stimulus_window)
val_ds = AlgonautsDataset(features_dir, fmri_dir, movies=movies_val, subject=subject, excluded_samples_start=excluded_samples_start, excluded_samples_end=excluded_samples_end, hrf_delay=hrf_delay, stimulus_window=stimulus_window)

In [14]:
print("Train dataset length: ", len(train_ds))
print("Validation dataset length: ", len(val_ds))

Train dataset length:  129516
Validation dataset length:  22924


# Start Building Your Model Here

The data is now prepared - you can begin implementing your model architecture and training pipeline below.

# Run inference on trained model

Friends Season 7 is the test set. Its corresponding fMRI data is withheld.

In [15]:
movies_test = ["friends-s07"] 
test_ds = AlgonautsDataset(features_dir, fmri_dir, movies=movies_test, subject=subject, excluded_samples_start=excluded_samples_start, excluded_samples_end=excluded_samples_end, hrf_delay=hrf_delay, stimulus_window=stimulus_window)
test_stimuli = test_ds.get_raw_stimuli()

In [16]:
for key_modality, value_modality in test_stimuli.items():
    print(f"\n{key_modality} features movie names and shape:")
    for key_movie, value_movie in value_modality.items():
        print(key_movie + " " + str(value_movie.shape))


visual features movie names and shape:
s07e01a (460, 8192)
s07e01b (494, 8192)
s07e02a (492, 8192)
s07e02b (526, 8192)
s07e03a (418, 8192)
s07e03b (452, 8192)
s07e04a (448, 8192)
s07e04b (482, 8192)
s07e05a (454, 8192)
s07e05b (488, 8192)
s07e06a (478, 8192)
s07e06b (513, 8192)
s07e07a (473, 8192)
s07e07b (507, 8192)
s07e08a (475, 8192)
s07e08b (509, 8192)
s07e09a (451, 8192)
s07e09b (485, 8192)
s07e10a (460, 8192)
s07e10b (494, 8192)
s07e11a (465, 8192)
s07e11b (499, 8192)
s07e12a (452, 8192)
s07e12b (487, 8192)
s07e13a (540, 8192)
s07e13b (574, 8192)
s07e14a (467, 8192)
s07e14b (501, 8192)
s07e15a (567, 8192)
s07e15b (601, 8192)
s07e16a (398, 8192)
s07e16b (402, 8192)
s07e16c (432, 8192)
s07e17a (433, 8192)
s07e17b (468, 8192)
s07e18a (477, 8192)
s07e18b (512, 8192)
s07e19a (437, 8192)
s07e19b (471, 8192)
s07e20a (454, 8192)
s07e20b (488, 8192)
s07e21a (430, 8192)
s07e21b (464, 8192)
s07e22a (472, 8192)
s07e22b (506, 8192)
s07e23a (481, 8192)
s07e23b (485, 8192)
s07e23c (485, 8192)


In [17]:
aligned_features_friends_s7 = align_features_and_fmri_samples_friends_s7(
    test_stimuli, fmri_dir)

# As an example, print the shape of the stimulus features of one episode for
# each subject
for key, val in aligned_features_friends_s7.items():
    episode_name = "s07e01a"
    example_episode_shape_visual = val[episode_name]['visual'].shape
    example_episode_shape_audio = val[episode_name]['audio'].shape
    example_episode_shape_language = val[episode_name]['language'].shape
    print(f"Subject: {key}")
    print(f"  Episode: {episode_name} - visual features shape: {example_episode_shape_visual}")
    print(f"  Episode: {episode_name} - audio features shape: {example_episode_shape_audio}")
    print(f"  Episode: {episode_name} - language features shape: {example_episode_shape_language}")
    print("-" * 40)

Aligning stimulus and fMRI features of the four subjects: 100%|██████████| 4/4 [00:08<00:00,  2.22s/it]

Subject: sub-01
  Episode: s07e01a - visual features shape: (460, 5, 8192)
  Episode: s07e01a - audio features shape: (460, 5, 128)
  Episode: s07e01a - language features shape: (460, 768)
----------------------------------------
Subject: sub-02
  Episode: s07e01a - visual features shape: (460, 5, 8192)
  Episode: s07e01a - audio features shape: (460, 5, 128)
  Episode: s07e01a - language features shape: (460, 768)
----------------------------------------
Subject: sub-03
  Episode: s07e01a - visual features shape: (460, 5, 8192)
  Episode: s07e01a - audio features shape: (460, 5, 128)
  Episode: s07e01a - language features shape: (460, 768)
----------------------------------------
Subject: sub-05
  Episode: s07e01a - visual features shape: (460, 5, 8192)
  Episode: s07e01a - audio features shape: (460, 5, 128)
  Episode: s07e01a - language features shape: (460, 768)
----------------------------------------


In [ ]:
submission_predictions = {}

# Loop through each subject
desc = "Predicting fMRI responses of each subject"
for sub, features in tqdm(aligned_features_friends_s7.items(), desc=desc):

    # Initialize the nested dictionary for each subject's predictions
    submission_predictions[sub] = {}

    # Loop through each Friends season 7 episode
    for epi, feat_epi in features.items():

        # convert the predictions to float32
        video = torch.tensor(feat_epi['visual']).cuda()
        audio = torch.tensor(feat_epi['audio']).cuda()
        text = torch.tensor(feat_epi['language']).cuda()
        fmri_pred = model(video, audio, text).detach().cpu().numpy() #TODO: Replace model with your trained model

        # Store formatted predictions in the nested dictionary
        submission_predictions[sub][epi] = fmri_pred

In [ ]:
# Select the saving directory
run_name = "model_name"
save_dir = Path('saved_preds/') / run_name
os.makedirs(save_dir, exist_ok=True)

# Save the predicted fMRI dictionary as a .npy file
output_file = save_dir / "fmri_predictions_friends_s7.npy"
np.save(output_file, submission_predictions)
print(f"Formatted predictions saved to: {output_file}")

# Zip the saved file for submission
zip_file = save_dir / "fmri_predictions_friends_s7.zip"  # Use / instead of +
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(output_file, os.path.basename(output_file))
print(f"Submission file successfully zipped as: {zip_file}")

### Upload the zip file to Codabench for eval.